## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Fortuna,US,40.60,-124.16,61.0,72,1,8.05,clear sky,2020-10-31 23:24:11
1,1,Busselton,AU,-33.65,115.33,59.0,79,100,1.97,light rain,2020-10-31 23:23:58
2,2,Brookhaven,US,31.58,-90.44,59.0,87,0,2.17,clear sky,2020-10-31 23:33:08
3,3,Albany,US,42.60,-73.97,36.0,83,0,5.41,clear sky,2020-10-31 23:21:30
4,4,Ushuaia,AR,-54.80,-68.30,48.2,93,90,5.82,light intensity drizzle,2020-10-31 23:33:08


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
6,6,Butaritari,KI,3.07,172.79,81.63,75,99,15.26,overcast clouds,2020-10-31 23:24:08
16,16,Carutapera,BR,-1.20,-46.02,78.13,81,25,6.58,scattered clouds,2020-10-31 23:32:00
20,20,Rikitea,PF,-23.12,-134.97,75.96,66,0,6.31,clear sky,2020-10-31 23:33:11
28,28,Natal,BR,-5.79,-35.21,78.80,83,20,9.17,few clouds,2020-10-31 23:24:34
30,30,Ixtapa,MX,20.70,-105.20,84.20,74,75,8.05,broken clouds,2020-10-31 23:33:12


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Butaritari,KI,81.63,overcast clouds,3.07,172.79,
16,Carutapera,BR,78.13,scattered clouds,-1.20,-46.02,
20,Rikitea,PF,75.96,clear sky,-23.12,-134.97,
28,Natal,BR,78.80,few clouds,-5.79,-35.21,
30,Ixtapa,MX,84.20,broken clouds,20.70,-105.20,
31,Hilo,US,86.00,scattered clouds,19.73,-155.09,
42,Butterworth,MY,77.00,scattered clouds,5.40,100.36,
43,Pizarro,CO,77.77,light rain,4.95,-77.37,
47,San Nicolas,PH,78.80,broken clouds,18.17,120.60,
50,Tautira,PF,86.00,scattered clouds,-17.73,-149.15,


In [12]:
len(hotel_df)

201

In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.


# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params = {
    "location": f"{lat},{lng}",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
    }

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
        
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): print("Hotel not found... skipping.")   
        

Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Butaritari,KI,81.63,overcast clouds,3.07,172.79,Isles Sunset Lodge
16,Carutapera,BR,78.13,scattered clouds,-1.20,-46.02,Carutapera
20,Rikitea,PF,75.96,clear sky,-23.12,-134.97,Rikitea
28,Natal,BR,78.80,few clouds,-5.79,-35.21,Natal
30,Ixtapa,MX,84.20,broken clouds,20.70,-105.20,Puerto Vallarta


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))